# [Output Widgets](https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html)

- Output widgets can capture and display stdout, stderr and rich output generated by IPython. You can also append output directly to an output widget, or clear it programmatically.

- After the widget is created, direct output to it using a context manager.

In [1]:
import ipywidgets as widgets

In [2]:
out1 = widgets.Output(layout={'border': '1px solid black'})
out1

Output(layout=Layout(border='1px solid black'))

In [3]:
with out1:
    for i in range(5):
        print(i, 'Hello world!')

In [4]:
out2 = widgets.Output(layout={'border': '1px dotted black'})
out2.append_stdout(display(widgets.IntSlider()))
out2

IntSlider(value=0)

Output(layout=Layout(border='1px dotted black'), outputs=({'output_type': 'stream', 'name': 'stdout', 'text': …

- Rich output can also be directed to the output area. Anything which displays nicely in a Jupyter notebook will also display well in the Output widget.

In [5]:
from IPython.display import YouTubeVideo

out3 = widgets.Output(layout={'border': '1px solid black'})
out3.append_stdout('Output appended with append_stdout')
out3.append_display_data(YouTubeVideo('eWzY2nGfkXk'))
out3

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'stream', 'name': 'stdout', 'text': '…

- You can clear outputs using a context manager, or by calling a widget's _clear_output_ method directly.

In [8]:
out = widgets.Output(layout={'border':'1px red solid'})
out.append_stdout("howdy")
out

Output(layout=Layout(border='1px red solid'), outputs=({'output_type': 'stream', 'name': 'stdout', 'text': 'ho…

In [13]:
out.clear_output()

### Capturing output from a function

In [12]:
@out.capture()
def function_with_captured_output():
    print('This goes into the output widget')
    raise Exception('As does this')

function_with_captured_output()

### Output widgets as foundation for interactions

In [14]:
a = widgets.IntSlider(description='a')
b = widgets.IntSlider(description='b')
c = widgets.IntSlider(description='c')

def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])

### Debugging callback errors with output widgets

- On some platforms, like JupyterLab, output generated by widget callbacks (for instance, functions attached to the .observe method on widget traits, or to the .on_click method on button widgets) are not displayed anywhere. Even on other platforms, it is unclear what cell this output should appear in. This can make debugging errors in callback functions more challenging.

- An effective tool for accessing the output of widget callbacks is to decorate the callback with an output widget’s capture method. You can then display the widget in a new cell to see the callback output.

In [15]:
debug_view = widgets.Output(layout={'border': '1px solid black'})

@debug_view.capture(clear_output=True)
def bad_callback(event):
    print('This is about to explode')
    return 1.0 / 0.0

button = widgets.Button(
    description='click me to raise an exception',
    layout={'width': '300px'}
)
button.on_click(bad_callback)
button

Button(description='click me to raise an exception', layout=Layout(width='300px'), style=ButtonStyle())

In [16]:
debug_view

Output(layout=Layout(border='1px solid black'), outputs=({'name': 'stdout', 'text': 'This is about to explode\…

### Logging module integration

- While using the .capture decorator works well for understanding and debugging single callbacks, it does not scale to larger applications. 

- One might use the logging module to print status information. However it is unclear where the logging output should go.

- A useful pattern is to create a custom handler that redirects logs to an output widget. The output widget can then be displayed in a new cell to monitor the application while it runs.

In [17]:
import ipywidgets as widgets
import logging

class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '100%',
            'height': '160px',
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout',
            'output_type': 'stream',
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs

    def show_logs(self):
        """ Show the logs """
        display(self.out)

    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()


logger = logging.getLogger(__name__)
handler = OutputWidgetHandler()
handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logger.addHandler(handler)
logger.setLevel(logging.INFO)

In [18]:
handler.show_logs()

Output(layout=Layout(border='1px solid black', height='160px', width='100%'))

In [19]:
handler.clear_logs()

### Widgets and background threads

- Jupyter’s display mechanism can be counter-intuitive when displaying output produced by background threads. A background thread’s output is printed to whatever cell the main thread is currently writing to. To see this directly, create a thread that repeatedly prints to standard out.

In [25]:
import threading, time, itertools

def run():
    for i in itertools.count(0):
        time.sleep(1)
        print('output from background {}'.format(i))

t = threading.Thread(target=run)
t.start()

output from background 0
output from background 1
output from background 2
output from background 3
output from background 4
output from background 5
output from background 6
output from background 7
output from background 8
output from background 9
output from background 10
output from background 11
output from background 12
output from background 13
output from background 14
output from background 15
output from background 16
output from background 17


- This always prints in the currently active cell, not the cell that started the background thread.

- This can lead to surprising behaviour in output widgets. During the time in which output is captured by the output widget, any output generated in the notebook, regardless of thread, will go into the output widget.

- The best way to avoid surprises is to never use an output widget’s context manager in a context where multiple threads generate output. Instead, we can pass an output widget to the function executing in a thread, and use append_display_data(), append_stdout(), or append_stderr() methods to append displayable output to the output widget.

In [ ]:
import threading
from IPython.display import display, HTML
import ipywidgets as widgets
import time

def thread_func(something, out):
    for i in range(1, 5):
        time.sleep(0.3)
        out.append_stdout('{} {} {}\n'.format(i, '**'*i, something))
    out.append_display_data(HTML("<em>All done!</em>"))

display('Display in main thread')
out = widgets.Output()
# Now the key: the container is displayed (while empty) in the main thread
display(out)

thread = threading.Thread(
    target=thread_func,
    args=("some text", out))
thread.start()

In [ ]:
thread.join()